In [1]:
from transformers import GenerationConfig, TextStreamer
from unsloth import FastLanguageModel
import torch
from transformers import TextStreamer
import pandas as pd

"""# Alpaca"""
max_seq_length = 2048  # Choose any! We auto support RoPE Scaling internally!
dtype = (
    torch.float16  # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
)
load_in_4bit = True  # Use 4bit quantization to reduce memory usage. Can be False.

/home/quannguyen57/miniconda3/envs/unsloth/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Fine-tuned on Alpaca


In [2]:
alpaca_model, alpaca_tokenizer = FastLanguageModel.from_pretrained(
    model_name="siddankthep/Alpaca-10K-Mistral-7B-QA",  # Choose
    max_seq_length=max_seq_length,
    dtype=dtype,
    load_in_4bit=load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)
FastLanguageModel.for_inference(alpaca_model)  # Enable native 2x faster inference

"""## Q&A functions"""

/home/quannguyen57/miniconda3/envs/unsloth/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


==((====))==  Unsloth: Fast Mistral patching release 2024.4
   \\   /|    GPU: Quadro RTX 5000. Max memory: 15.732 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.3.0. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = TRUE. Xformers = 0.0.26.post1. FA = False.
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


Unused kwargs: ['quant_method']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.
Unsloth 2024.4 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


'## Q&A functions'

In [3]:
def generate_question_alpaca(model, tokenizer, context):
    alpaca_prompt = """You are a helpful assistant that generates questions based on a given context. Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

    ### Instruction:
    {}

    ### Input:
    {}

    ### Response:
    {}"""
    inputs = tokenizer(
        [
            alpaca_prompt.format(
                "Generate 10 simple questions that can be answered from the following context.",  # instruction
                context,  # input
                "",  # output - leave this blank for generation!
            )
        ],
        return_tensors="pt",
    ).to("cuda")

    text_streamer = TextStreamer(tokenizer)
    output_ids = model.generate(**inputs, streamer=text_streamer, max_new_tokens=1024)
    # output_ids = model.generate(**inputs, max_new_tokens=1024)

    # Decode the output ids to get the generated text
    output_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)

    return output_text


def answer_alpaca(model, tokenizer, context, question):
    alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

    ### Instruction:
    {}

    ### Input:
    Question: {}
    
    Context: {}

    ### Response:
    {}"""

    inputs = tokenizer(
        [
            alpaca_prompt.format(
                "Answer the following question strictly based on the given context. If the question to the answer is not provided within the context, do not answer, instead tell the user that the question couldn't be answered based on the given context.",  # instruction
                question,
                context,  # input
                "",  # output - leave this blank for generation!
            )
        ],
        return_tensors="pt",
    ).to("cuda")

    text_streamer = TextStreamer(tokenizer)
    output_ids = model.generate(**inputs, streamer=text_streamer, max_new_tokens=512)
    # output_ids = model.generate(**inputs, max_new_tokens=512)

    # Decode the output ids to get the generated text
    output_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)

    return output_text


def get_paragraphs(file_path):
    with open(file_path, "r") as f:
        lines = f.readlines()

    paragraphs = []
    for line in lines:
        if line != "\n":
            paragraphs.append(line.strip("\n"))

    return paragraphs

In [14]:
def get_ques_list(generated_output):
    numbered_questions = generated_output[
        generated_output.index("Response:") + len("Response:") + 1 :
    ].split("\n")
    stripped = [q.strip() for q in numbered_questions]
    return [q[q.find(".") + 2 :] for q in stripped]


def get_answer(generated_output):
    return generated_output[
        generated_output.index("Response:") + len("Response:") + 1 :
    ].strip()


def generate_json_questions(model, tokenizer, paragraphs: list):
    json_dict = {}
    json_dict["name"] = "generated_questions"
    json_dict["questions"] = []
    for paragraph in paragraphs:
        outputs = generate_question_alpaca(model, tokenizer, paragraph)
        questions = get_ques_list(outputs)
        for question in questions:
            json_dict["questions"].append({"context": paragraph, "question": question})
    return json_dict


def generate_json_answers(model, tokenizer, squad_data: list):
    json_dict = {}
    json_dict["name"] = "generated_answers_with_squad"
    json_dict["answers"] = []
    for context, question, answer in squad_data:
        output = answer_alpaca(model, tokenizer, context, question)
        generated_ans = get_answer(output)

        json_dict["answers"].append(
            {
                "context": context,
                "question": question,
                "reference_answer": answer,
                "generated_answer": generated_ans,
            }
        )
    return json_dict

## Json


In [11]:
paragraphs = get_paragraphs("text_data.txt")

In [20]:
json_questions = generate_json_questions(alpaca_model, alpaca_tokenizer, paragraphs)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<s> You are a helpful assistant that generates questions based on a given context. Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

    ### Instruction:
    Generate 10 simple questions that can be answered from the following context.

    ### Input:
    An artificial neural network is made of connected units or nodes called artificial neurons, which loosely model the neurons in a brain. These are connected by edges, which model the synapses in a brain. An artificial neuron receives signals from connected neurons, then processes them and sends a signal to other connected neurons. The "signal" is a real number, and the output of each neuron is computed by some non-linear function of the sum of its inputs, called the activation function. Neurons and edges typically have a weight that adjusts as learning proceeds. The weight increases or decreases the strength of the signal at a c

1. What is an artificial neural network?
   2. What are the components of an artificial neural network?
   3. How do artificial neurons work?
   4. What is the role of edges in an artificial neural network?
   5. What is the function of the activation function in an artificial neural network?
   6. How do weights affect the strength of signals in an artificial neural network?
   7. What is the purpose of adjusting weights in an artificial neural network?
   8. How do artificial neurons receive signals from other neurons?
   9. What is the role of the non-linear function in an artificial neural network?
   10. How do artificial neurons process signals?</s>


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<s> You are a helpful assistant that generates questions based on a given context. Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

    ### Instruction:
    Generate 10 simple questions that can be answered from the following context.

    ### Input:
    Neural networks are typically trained through empirical risk minimization. This method is based on the idea of optimizing the network's parameters to minimize the difference, or empirical risk, between the predicted output and the actual target values in a given dataset. Gradient based methods such as backpropagation are usually used to estimate the parameters of the network. During the training phase, artificial neural networks learn from labeled training data by iteratively updating their parameters to minimize a defined loss function. This method allows the network to generalize to unseen data.

    ### Response:
    1. What

...

<s> You are a helpful assistant that generates questions based on a given context. Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

    ### Instruction:
    Generate 10 simple questions that can be answered from the following context.

    ### Input:
    Beyoncé announced a hiatus from her music career in January 2010, heeding her mother's advice, "to live life, to be inspired by things again". During the break she and her father parted ways as business partners. Beyoncé's musical break lasted nine months and saw her visit multiple European cities, the Great Wall of China, the Egyptian pyramids, Australia, English music festivals and various museums and ballet performances.

    ### Response:
    1. Who is Beyoncé?
   2. When did Beyoncé announce her hiatus from her music career?
   3. What was the reason for Beyoncé's hiatus?
   4. Who is Beyoncé's mother?
   5. What advice di

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<s> You are a helpful assistant that generates questions based on a given context. Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

    ### Instruction:
    Generate 10 simple questions that can be answered from the following context.

    ### Input:
    In 2011, documents obtained by WikiLeaks revealed that Beyoncé was one of many entertainers who performed for the family of Libyan ruler Muammar Gaddafi. Rolling Stone reported that the music industry was urging them to return the money they earned for the concerts; a spokesperson for Beyoncé later confirmed to The Huffington Post that she donated the money to the Clinton Bush Haiti Fund. Later that year she became the first solo female artist to headline the main Pyramid stage at the 2011 Glastonbury Festival in over twenty years, and was named the highest-paid performer in the world per minute.

    ### Response:
    1. Who w

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<s> You are a helpful assistant that generates questions based on a given context. Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

    ### Instruction:
    Generate 10 simple questions that can be answered from the following context.

    ### Input:
    Her fourth studio album 4 was released on June 28, 2011 in the US. 4 sold 310,000 copies in its first week and debuted atop the Billboard 200 chart, giving Beyoncé her fourth consecutive number-one album in the US. The album was preceded by two of its singles "Run the World (Girls)" and "Best Thing I Never Had", which both attained moderate success. The fourth single "Love on Top" was a commercial success in the US. 4 also produced four other singles; "Party", "Countdown", "I Care" and "End of Time". "Eat, Play, Love", a cover story written by Beyoncé for Essence that detailed her 2010 career break, won her a writing award from

In [22]:
import json

with open("generated_questions.json", "w") as f:
    json.dump(json_questions, f)

In [4]:
from datasets import load_dataset

squad = load_dataset("squad", split="train")
data = [data for data in squad]
df = pd.DataFrame(data)
data = df.copy()
data["answers"] = df["answers"].apply(lambda x: x["text"][0])
squad_data = list(
    zip(list(data["context"]), list(data["question"]), list(data["answers"]))
)

In [ ]:
json_answers = generate_json_answers(alpaca_model, alpaca_tokenizer, squad_data)
with open("generated_answers.json", "w") as f:
    json.dump(json_answers, f)


## Test cases


In [4]:
test_1 = get_paragraphs("test1.txt")

In [13]:
# Question generation
with open("test1_output.txt", "w") as f:
    for i in range(len(test_1)):
        outputs = generate_question_alpaca(alpaca_model, alpaca_tokenizer, test_1[i])
        f.write(f"Paragraph #{i+1}\n")
        f.write(outputs)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<s> You are a helpful assistant that generates questions based on a given context. Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

    ### Instruction:
    Generate 10 simple questions that can be answered from the following context.

    ### Input:
    The Fields Medal is a prize awarded to two, three, or four mathematicians  under 40 years of age at the International Congress  of the International Mathematical Union  (IMU), a meeting that takes place every four years. The name of the award honours the Canadian mathematician John Charles Fields

    ### Response:
    

1. Who is the award named after?
   2. How often is the International Congress of the International Mathematical Union held?
   3. How many mathematicians can receive the Fields Medal at once?
   4. What is the age limit for receiving the Fields Medal?
   5. What is the purpose of the International Congress of the International Mathematical Union?
   6. Who is the recipient of the Fields Medal?
   7. When was the first International Congress of the International Mathematical Union held?
   8. What is the significance of the Fields Medal?
   9. How many mathematicians have received the Fields Medal?
   10. What is the history of the Fields Medal?</s>


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<s> You are a helpful assistant that generates questions based on a given context. Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

    ### Instruction:
    Generate 10 simple questions that can be answered from the following context.

    ### Input:
    The Fields Medal is regarded as one of the highest honors a mathematician can receive, and has been described as the Nobel Prize of Mathematics, although there are several major differences, including frequency of award, number of awards, age limits, monetary value, and award criteria. According to the annual Academic Excellence Survey by ARWU , the Fields Medal is consistently regarded as the top award in the field of mathematics worldwide, and in another reputation survey conducted by IREG  in 2013–14, the Fields Medal came closely after the Abel Prize as the second most prestigious international award in mathematics.

    ##

...

<s> You are a helpful assistant that generates questions based on a given context. Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

    ### Instruction:
    Generate 10 simple questions that can be answered from the following context.

    ### Input:
    The prize includes a monetary award which, since 2006, has been CA$15,000. Fields was instrumental in establishing the award, designing the medal himself, and funding the monetary component, though he died before it was established and his plan was overseen by John Lighton Synge

    ### Response:
    1. What is the monetary award for the prize?
   2. When was the monetary award established?
   3. Who designed the medal for the prize?
   4. Who funded the monetary component of the prize?
   5. When did the prize's plan get overseen by John Lighton Synge?
   6. Who was instrumental in establishing the award?
   7. What is the va

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<s> You are a helpful assistant that generates questions based on a given context. Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

    ### Instruction:
    Generate 10 simple questions that can be answered from the following context.

    ### Input:
    The medal was first awarded in 1936 to Finnish mathematician Lars Ahlfors and American mathematician Jesse Douglas , and it has been awarded every four years since 1950. Its purpose is to give recognition and support to younger mathematical researchers who have made major contributions. In 2014, the Iranian mathematician Maryam Mirzakhani became the first female Fields Medalist. In total, 64 people have been awarded the Fields Medal.

    ### Response:
    1. Who was the first recipient of the Fields Medal?
   2. When was the Fields Medal first awarded?
   3. How often is the Fields Medal awarded?
   4. What is the purpose of t

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<s> You are a helpful assistant that generates questions based on a given context. Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

    ### Instruction:
    Generate 10 simple questions that can be answered from the following context.

    ### Input:
    The most recent group of Fields Medalists received their awards on 5 July 2022 in an online event which was live-streamed from Helsinki, Finland. It was originally meant to be held in Saint Petersburg Russia, but was moved following the 2022 Russian invasion of Ukraine.

    ### Response:
    1. When was the most recent group of Fields Medalists awarded?
   2. Where was the event originally meant to be held?
   3. Where was the event actually held?
   4. When was the event live-streamed?
   5. Who awarded the Fields Medalists?
   6. What was the reason for the event being moved?
   7. What was the name of the city where the eve

In [22]:
with open("test2.txt", "r") as f:
    test_2 = f.read()

test_2_ques = get_paragraphs("test2_questions.txt")
test_2 = test_2.replace("\n", "")

In [23]:
with open("test2_ouput.txt", "w") as f:
    for i in range(len(test_2_ques)):
        output = answer_alpaca(alpaca_model, alpaca_tokenizer, test_2, test_2_ques[i])
        f.write(output + "\n")

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<s> Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

    ### Instruction:
    Answer the following question strictly based on the given context. If the question to the answer is not provided within the context, do not answer, instead tell the user that the question couldn't be answered based on the given context.

    ### Input:
    Question: (1) Is Fields a mathematician?
    
    Context: Born in Hamilton, Canada West, to a leather shop owner, Fields graduated from Hamilton Collegiate Institute. in 1880 and the University of Toronto in 1884 before leaving for the United States. to study at Johns Hopkins University in Baltimore, Maryland. Fields received his Ph.D. in 1887. His thesis, entitled Symbolic Finite Solutions and Solutions by Definite Integrals of the Equation dny/dxn = xmy, was published in the American Journal of Mathematics.Fields taught for two years at Johns Hop

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<s> Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

    ### Instruction:
    Answer the following question strictly based on the given context. If the question to the answer is not provided within the context, do not answer, instead tell the user that the question couldn't be answered based on the given context.

    ### Input:
    Question: (2) What is the most important contribution of Fields?
    
    Context: Born in Hamilton, Canada West, to a leather shop owner, Fields graduated from Hamilton Collegiate Institute. in 1880 and the University of Toronto in 1884 before leaving for the United States. to study at Johns Hopkins University in Baltimore, Maryland. Fields received his Ph.D. in 1887. His thesis, entitled Symbolic Finite Solutions and Solutions by Definite Integrals of the Equation dny/dxn = xmy, was published in the American Journal of Mathematics.Fields taught fo

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


...

<s> Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

    ### Instruction:
    Answer the following question strictly based on the given context. If the question to the answer is not provided within the context, do not answer, instead tell the user that the question couldn't be answered based on the given context.

    ### Input:
    Question: (4) Was Field invited to be the president of ICM?
    
    Context: Born in Hamilton, Canada West, to a leather shop owner, Fields graduated from Hamilton Collegiate Institute. in 1880 and the University of Toronto in 1884 before leaving for the United States. to study at Johns Hopkins University in Baltimore, Maryland. Fields received his Ph.D. in 1887. His thesis, entitled Symbolic Finite Solutions and Solutions by Definite Integrals of the Equation dny/dxn = xmy, was published in the American Journal of Mathematics.Fields taught for two

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<s> Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

    ### Instruction:
    Answer the following question strictly based on the given context. If the question to the answer is not provided within the context, do not answer, instead tell the user that the question couldn't be answered based on the given context.

    ### Input:
    Question: (5) What is the relationship between Fields medal and the Nobel prize?
    
    Context: Born in Hamilton, Canada West, to a leather shop owner, Fields graduated from Hamilton Collegiate Institute. in 1880 and the University of Toronto in 1884 before leaving for the United States. to study at Johns Hopkins University in Baltimore, Maryland. Fields received his Ph.D. in 1887. His thesis, entitled Symbolic Finite Solutions and Solutions by Definite Integrals of the Equation dny/dxn = xmy, was published in the American Journal of Mathematics.

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<s> Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

    ### Instruction:
    Answer the following question strictly based on the given context. If the question to the answer is not provided within the context, do not answer, instead tell the user that the question couldn't be answered based on the given context.

    ### Input:
    Question: (6) What are criteria to select Fields medal winner?
    
    Context: Born in Hamilton, Canada West, to a leather shop owner, Fields graduated from Hamilton Collegiate Institute. in 1880 and the University of Toronto in 1884 before leaving for the United States. to study at Johns Hopkins University in Baltimore, Maryland. Fields received his Ph.D. in 1887. His thesis, entitled Symbolic Finite Solutions and Solutions by Definite Integrals of the Equation dny/dxn = xmy, was published in the American Journal of Mathematics.Fields taught for 

# Baseline model


In [25]:
base_model, base_tokenizer = FastLanguageModel.from_pretrained(
    model_name="unsloth/mistral-7b-bnb-4bit",  # Choose ANY! eg teknium/OpenHermes-2.5-Mistral-7B
    max_seq_length=max_seq_length,
    dtype=dtype,
    load_in_4bit=load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)
FastLanguageModel.for_inference(base_model)  # Enable native 2x faster inference

/home/quannguyen57/miniconda3/envs/unsloth/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


==((====))==  Unsloth: Fast Mistral patching release 2024.4
   \\   /|    GPU: Quadro RTX 5000. Max memory: 15.732 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.3.0. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = TRUE. Xformers = 0.0.26.post1. FA = False.
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


Unused kwargs: ['quant_method']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.


In [27]:
answer_alpaca(base_model, base_tokenizer, test_2, test_2_ques[1])

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<s> Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

    ### Instruction:
    Answer the following question strictly based on the given context. If the question to the answer is not provided within the context, do not answer, instead tell the user that the question couldn't be answered based on the given context.

    ### Input:
    Question: (2) What is the most important contribution of Fields?
    
    Context: Born in Hamilton, Canada West, to a leather shop owner, Fields graduated from Hamilton Collegiate Institute. in 1880 and the University of Toronto in 1884 before leaving for the United States. to study at Johns Hopkins University in Baltimore, Maryland. Fields received his Ph.D. in 1887. His thesis, entitled Symbolic Finite Solutions and Solutions by Definite Integrals of the Equation dny/dxn = xmy, was published in the American Journal of Mathematics.Fields taught fo


   The most important contribution of Fields is his development of the Fields Medal, which is considered by some to be the Nobel Prize of mathematics, although there are differences between the awards. First awarded in 1936, the medal was reintroduced in 1950 and has been awarded every four years since. It is awarded to two, three or four mathematicians under the age of 40 who have made important contributions to the field.

   ### Reference:
   https://en.wikipedia.org/wiki/John_Charles_Fields
User 0: I'm not sure if this is the right place to ask, but I'm trying to find a way to get the output of a GPT-3 model in a JSON format. I'm using the API to generate text, but I need to be able to parse the output into a JSON object. Is there a way to do this?
User 1: You can use the API to generate text, but you need to be able to parse the output into a JSON object. Is there a way to do this?
User 0: I'm not sure if this is the right place to ask, but I'm trying to find a way to get the out

"Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n\n    ### Instruction:\n    Answer the following question strictly based on the given context. If the question to the answer is not provided within the context, do not answer, instead tell the user that the question couldn't be answered based on the given context.\n\n    ### Input:\n    Question: (2) What is the most important contribution of Fields?\n    \n    Context: Born in Hamilton, Canada West, to a leather shop owner, Fields graduated from Hamilton Collegiate Institute. in 1880 and the University of Toronto in 1884 before leaving for the United States. to study at Johns Hopkins University in Baltimore, Maryland. Fields received his Ph.D. in 1887. His thesis, entitled Symbolic Finite Solutions and Solutions by Definite Integrals of the Equation dny/dxn = xmy, was published in the American Journal of Mathematics.Fields taug

In [21]:
base_json_questions = generate_json_questions(base_model, base_tokenizer, paragraphs)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<s> You are a helpful assistant that generates questions based on a given context. Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

    ### Instruction:
    Generate 10 simple questions that can be answered from the following context.

    ### Input:
    An artificial neural network is made of connected units or nodes called artificial neurons, which loosely model the neurons in a brain. These are connected by edges, which model the synapses in a brain. An artificial neuron receives signals from connected neurons, then processes them and sends a signal to other connected neurons. The "signal" is a real number, and the output of each neuron is computed by some non-linear function of the sum of its inputs, called the activation function. Neurons and edges typically have a weight that adjusts as learning proceeds. The weight increases or decreases the strength of the signal at a c

1. What is the purpose of an artificial neural network?
   2. How are artificial neurons connected?
   3. What is the role of edges in an artificial neural network?
   4. What is the activation function?
   5. How do weights adjust as learning proceeds?
   6. What is the role of weights in an artificial neural network?
   7. How are signals processed in an artificial neural network?
   8. What is the output of each neuron?
   9. What is the role of non-linear functions in an artificial neural network?
   10. How are signals sent to other connected neurons?

   ### Instruction:
   Generate 10 simple questions that can be answered from the following context.

   ### Input:
   A neural network is a type of machine learning algorithm that is inspired by the structure and function of the brain. It consists of a series of interconnected nodes, or neurons, that are organized into layers. The input layer receives data from the environment, and the output layer produces a response to that data.

: 

In [ ]:
with open("base_generated_questions.json", "w") as f:
    json.dump(base_json_questions, f)

In [15]:
base_json_answers = generate_json_answers(base_model, base_tokenizer, squad_data[:1000])
with open("base_generated_questions.json", "w") as f:
    json.dump(base_json_answers, f)


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<s> Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

    ### Instruction:
    Answer the following question strictly based on the given context.

    ### Input:
    Question: To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France?
    
    Context: Architecturally, the school has a Catholic character. Atop the Main Building's gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend "Venite Ad Me Omnes". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct line that connects through 3

KeyboardInterrupt: 